## Step 1

In [1]:
import csv
import os
import shutil

In [2]:
# definisco la prima funzione, che mi permetterà di creare le cartelle necessarie se non dovessero essere già presenti
def directories_check(path):
    subfolders = ["audio", "docs", "images"]
    for folder in subfolders:
        # ho utilizzato os.makedirs con keyword argument "exist_ok" perché  più pulito e leggibile rispetto ad una clausola try/except
        os.makedirs(os.path.join(path, folder), exist_ok=True)
        # in tutto il programma utilizzo os.path.join perché in questo modo modificando opportunatamente la variabile path il programma funziona su
        # qualsiasi sistema operativo

In [3]:
def files_handling(path):
    # itero su os.listdir(path) e tramite una list comprehension creo una lista contenente i nomi di tutti i file all'interno del path specificato
    files_list = [name for name in os.listdir(path) if os.path.isfile(os.path.join(path,name))]
    # ho creato delle liste di stringhe per le estensioni in modo da poter inserire eventualmente ulteriori estensioni per ogni tipo
    images_ext = [".png", ".jpg", ".jpeg"]
    audio_ext = [".mp3"]
    docs_ext = [".txt", ".odt"]

    # apro il file recap.csv in modalità append per fare in modo da creare il file se non dovesse esistere ed allo stesso tempo non sovrascrivelo nel caso in cui 
    # esista già, con parametro newline per evitare che ci sia un doppio spazio fra le righe
    with open(os.path.join(path,"recap.csv"), "a", newline = "") as csv_file:
        writer = csv.writer(csv_file)
        # if statement che controlla se il file "recap.csv" sia già presente, così da non inserire nuovamente l'header
        if "recap.csv" not in files_list:
            writer.writerow(["Filename","Type","Size(B)"])

        # nel for loop uso la funzione sorted per iterare in ordine alfabetico sui files
        for file in sorted(files_list):
            # multiple assignment per dividere il nome di ciascun file dalla sua estensione con os.path.splitext
            fileName,fileExtension = os.path.splitext(file)
            # ottengo la dimensione del file con os.path.getsize, ho cercato di utilizzare nomi appropriati per rendere il programma più leggibile
            fileSize = os.path.getsize(os.path.join(path,file))
            # controllo la tipologia di ogni file sfruttando le liste di stringhe create prima ed il membership operator
            # dopo il controllo stampo le informazioni, le riporto all'interno di recap.csv e sposto il file nella sua cartella
            if fileExtension in images_ext:
                print(f"{fileName} type: image size: {fileSize}B")
                writer.writerow([fileName, "image", fileSize])
                shutil.move(os.path.join(path,file), os.path.join(path,"images"))
            elif fileExtension in audio_ext:
                print(f"{fileName} type: audio size: {fileSize}B")
                writer.writerow([fileName, "audio", fileSize])
                shutil.move(os.path.join(path,file), os.path.join(path,"audio"))
            elif fileExtension in docs_ext:
                print(f"{fileName} type: document size: {fileSize}B")
                writer.writerow([fileName, "docs", fileSize])
                shutil.move(os.path.join(path,file), os.path.join(path,"docs"))

In [4]:
# definisco una funzione main() con la variabile locale path al suo interno, ho organizzato il programma in maniera che per farlo funzionare su
# un altro computer sia necessario cambiare solo questa variabile. In questo caso la funzione main() non mi sembrava necessaria ma ho voluto aggiungerla
# anche per evitare di confondermi tra la variabile locale path e quella globale image_path che mi è servita nello step 3
def main():
    path = r"./files"
    directories_check(path)
    files_handling(path)

In [5]:
main()

bw type: image size: 94926B
ciao type: document size: 12B
daffodil type: image size: 24657B
eclipse type: image size: 64243B
pippo type: document size: 8299B
song1 type: audio size: 1087849B
song2 type: audio size: 764176B
trump type: image size: 10195B


## Step 3

In [6]:
import numpy as np
import os
from PIL import Image
from tabulate import tabulate

# L'ultima parte del programma non è inserita in una funzione perché avrei dovuto passare troppi parametri alle altre funzioni e mi sembrava confusionario,
# quindi ho definito delle variabili globali e le ho utilizzate all'interno delle funzioni. 

In [7]:
def deal_with_colors():
    # ottengo la media dei valori di ogni livello di colore utilizzando il keyword argument "axis = 0" per lavorare sulle colonne, lo utilzzo la prima volta
    # per ottenere un array con shape (275, 3) ed una seconda volta per ottenere una shape di (3,) contenente la media di ciascun livello di colore
    colours_avg = img_array.mean(axis = 0).mean(axis = 0)
    # assegno ad ogni chiave del dizionario un valore corrispondente alla rispettiva media, con la funzione round per arrotondare a due decimali
    RGBA["R"] = round(colours_avg[0], 2)
    RGBA["G"] = round(colours_avg[1], 2)
    RGBA["B"] = round(colours_avg[2], 2)
    # controllo se l'immagine è RGBA e nel caso assegno un valore che indichi la media di alpha
    if len(colours_avg) == 4:
        RGBA["ALPHA"] = round(colours_avg[3], 2)

In [8]:
# definisco le informazioni relative ad ogni immagine e creo una lista di stringhe da trasferire all'interno della lista table
def adding_info():
    info = [imgName, height, width, RGBA["grayscale"], RGBA["R"], RGBA["G"], RGBA["B"], RGBA["ALPHA"]]
    table.append(info)

In [9]:
# anche in questo caso la variabile image_path è l'unica da cambiare per utilizzare il programma su un altro computer
image_path = r"./files/images"
# ho utilizzato una clasuola try/except nel caso in cui il path dovesse essere sbagliato o la cartella dovesse essere inesistente
try :
    files_list = [name for name in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, name))]
except FileNotFoundError:
    folder = os.path.basename(image_path)
    # In caso di errore FileNotFoundError stampo le informazioni che potrebbero essere più utili per risolverlo
    print(f"""Invalid path, please check that the path : "{image_path}" is valid and that the folder "{folder}" exists""")
# creo una lista vuota table, che diventerà una lista di liste e che al termine verrà passata alla funzione tabulate
table = []
header = ["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]
# creo l'header con i parametri necessari e lo inserisco come primo elemento della lista table
table.append(header)

In [10]:
# itero sulla lista files_list e, dopo aver aperto l'immagine con la funzione Image e  passato a numpy.array l'immagine per trasformarla in un array,
# definisco delle variabili globali contenti le informazioni relative ad ogni immagine (array,dimensione, shape, altezza, larghezza)
for filename in files_list:
    imgName, imgExtension = os.path.splitext(filename)
    img = Image.open(os.path.join(image_path,filename))
    img_array = np.array(img)
    dimensions = img_array.ndim
    shape = img_array.shape
    height = shape[0]
    width = shape[1]
    # Ho utilizzato un dizionario in maniera da avere tutti i parametri impostati a zero di default, per poi cambiarli in base al contenuto di ogni immagine
    RGBA = {"grayscale" : 0, "R" : 0, "G" : 0, "B" : 0, "ALPHA" : 0}
    # if statement per distinguere tra immagini in bianco e nero e quelle a colori
    if dimensions == 2:
        # modifico l'unico parametro dell'immagine in scala di grigio, utilizzando il metodo mean() del numpy array, ed utilizzo round per arrotondare il risultato
        RGBA["grayscale"] = round(img_array.mean(), 2)
        adding_info()
    # la distinzione fra RGB ed RGBA viene fatta all'interno della funzione deal_with_colors() quindi qui passo semplicemente l'array alla funzione
    elif dimensions == 3:
        deal_with_colors()
        adding_info()

In [11]:
# stampo la lista table completa dopo averla passata alla funzione tabulate
print(tabulate(table))

--------  ------  -----  ---------  ------  ------  -----  ------
name      height  width  grayscale  R       G       B      ALPHA
bw        512     512    21.48      0       0       0      0
daffodil  500     335    0          109.25  85.56   4.97   0
eclipse   256     256    0          109.05  109.52  39.85  133.59
trump     183     275    0          97.01   98.99   90.92  0
--------  ------  -----  ---------  ------  ------  -----  ------
